In [1]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from dash import Dash, dcc, html, Input, Output  # pip install dash
from preprocess import preprocess
import os.path

app = JupyterDash(__name__)


In [2]:
# preprocess the data; if the data has been preprocessed, just read the preprocessed data
file_name = "atussum_0321.csv"
processed_file_name = "processed_atussum_0321.csv"

is_processed_file_exists = os.path.exists(processed_file_name)
if is_processed_file_exists:
    df = pd.read_csv(processed_file_name)
else:
    df = preprocess(file_name)
    df.to_csv(processed_file_name)


app.layout = html.Div([

    html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),

    html.Div([
        "Select some categories",
        dcc.Dropdown(id="slct_ctgr",
                 options=[
                     {"label": "labor_stat", "value": 0},
                     {"label": "edu_level", "value": 1},
                     {"label": "sex", "value": 2}],
                 multi=True,
                 value=[0,1],
                 style={'width': "40%"}
                 ),
    ]),
    

    html.Div(id='output_container', children=[]),
    html.Br(),

    dcc.Graph(id='my_bee_map', figure={})
    


])

# @app.callback(
#     Output('ctgr-output-container', 'children'),
#     Input('slct_ctgr', 'value')
# )
# def update_output(value):
#     return f'You have selected labor force {value}'
# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_bee_map', component_property='figure')],
    [Input(component_id='slct_ctgr', component_property='value')]
)
def update_graph(option_slctd):
    if(len(option_slctd) == 2):
        print(option_slctd)
        print(type(option_slctd))
        my_dist = ['TELFS','PEEDUCA','TESEX']
        container = "The ctgr chosen was: {}".format(option_slctd)
        cnt = df.groupby([my_dist[option_slctd[0]], my_dist[option_slctd[1]]]).size()
        cnt.to_csv("result.csv")
        cnt =  pd.read_csv("result.csv")
        cnt.rename(columns={'0':'count'}, inplace=True)
        print(cnt)

        # Plotly Express
        fig = px.scatter(cnt, x=my_dist[option_slctd[0]], y=my_dist[option_slctd[1]],size='count',log_x=True, size_max=60)
        fig.update_xaxes(type='category')
        fig.update_yaxes(type='category')
        return container, fig
    elif(len(option_slctd) == 3):
        print(option_slctd)
        print(type(option_slctd))
        my_dist = ['TELFS','PEEDUCA','TESEX']
        container = "The ctgr chosen was: {}".format(option_slctd)
        cnt = df.groupby([my_dist[option_slctd[0]], my_dist[option_slctd[1]],my_dist[option_slctd[2]]]).size()
        cnt.to_csv("result.csv")
        cnt =  pd.read_csv("result.csv")
        cnt.rename(columns={'0':'count'}, inplace=True)
        # Plotly Express
        fig = px.scatter(cnt, x=my_dist[option_slctd[0]], y=my_dist[option_slctd[1]],size='count',log_x=True, size_max=60,
                        color=my_dist[option_slctd[2]],color_discrete_sequence=px.colors.qualitative.Antique)
        fig.update_xaxes(type='category')
        fig.update_yaxes(type='category')
        return container, fig
    else:
        container = "The ctgr chosen was: {}".format(option_slctd)
        cnt = df.groupby(['PEEDUCA', 'TESEX']).size()
        cnt.to_csv("result.csv")
        cnt =  pd.read_csv("result.csv")
        cnt.rename(columns={'0':'count'}, inplace=True)
        fig = px.scatter(cnt, x='PEEDUCA', y='TESEX',size='count',log_x=True, size_max=60)
        fig.update_xaxes(type='category')
        fig.update_yaxes(type='category')
        return container, fig

In [3]:
# cnt = df.groupby(['PEEDUCA', 'TESEX']).size()
# cnt.to_csv("result.csv")
# cnt =  pd.read_csv("result.csv")
# #cnt
# fig = px.scatter(cnt, x='PEEDUCA', y='TESEX',size='0',log_x=True, size_max=60)
# fig.show()

In [4]:
# # -- Import and clean data (importing csv into pandas)
# # df = pd.read_csv("intro_bees.csv")
# df = pd.read_csv("https://raw.githubusercontent.com/Coding-with-Adam/Dash-by-Plotly/master/Other/Dash_Introduction/intro_bees.csv")

# df = df.groupby(['State', 'ANSI', 'Affected by', 'Year', 'state_code'])[['Pct of Colonies Impacted']].mean()
# df.reset_index(inplace=True)
# print(df[:5])

# # ------------------------------------------------------------------------------
# # App layout
# app.layout = html.Div([

#     html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),

#     dcc.Dropdown(id="slct_year",
#                  options=[
#                      {"label": "2015", "value": 2015},
#                      {"label": "2016", "value": 2016},
#                      {"label": "2017", "value": 2017},
#                      {"label": "2018", "value": 2018}],
#                  multi=False,
#                  value=2015,
#                  style={'width': "40%"}
#                  ),

#     html.Div(id='output_container', children=[]),
#     html.Br(),

#     dcc.Graph(id='my_bee_map', figure={})

# ])


# # ------------------------------------------------------------------------------
# # Connect the Plotly graphs with Dash Components
# @app.callback(
#     [Output(component_id='output_container', component_property='children'),
#      Output(component_id='my_bee_map', component_property='figure')],
#     [Input(component_id='slct_year', component_property='value')]
# )
# def update_graph(option_slctd):
#     print(option_slctd)
#     print(type(option_slctd))

#     container = "The year chosen by user was: {}".format(option_slctd)

#     dff = df.copy()
#     dff = dff[dff["Year"] == option_slctd]
#     dff = dff[dff["Affected by"] == "Varroa_mites"]

#     # Plotly Express
#     fig = px.choropleth(
#         data_frame=dff,
#         locationmode='USA-states',
#         locations='state_code',
#         scope="usa",
#         color='Pct of Colonies Impacted',
#         hover_data=['State', 'Pct of Colonies Impacted'],
#         color_continuous_scale=px.colors.sequential.YlOrRd,
#         labels={'Pct of Colonies Impacted': '% of Bee Colonies'},
#         template='plotly_dark'
#     )

#     # Plotly Graph Objects (GO)
#     # fig = go.Figure(
#     #     data=[go.Choropleth(
#     #         locationmode='USA-states',
#     #         locations=dff['state_code'],
#     #         z=dff["Pct of Colonies Impacted"].astype(float),
#     #         colorscale='Reds',
#     #     )]
#     # )
#     #
#     # fig.update_layout(
#     #     title_text="Bees Affected by Mites in the USA",
#     #     title_xanchor="center",
#     #     title_font=dict(size=24),
#     #     title_x=0.5,
#     #     geo=dict(scope='usa'),
#     # )

#     return container, fig

In [5]:
# # assume you have a "long-form" data frame
# # see https://plotly.com/python/px-arguments/ for more options
# df = pd.DataFrame({
#     "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
#     "Amount": [4, 1, 2, 2, 4, 5],
#     "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
# })

# fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

# app.layout = html.Div(children=[
#     html.H1(children='Hello Dash'),

#     html.Div(children='''
#         Dash: A web application framework for your data.
#     '''),

#     dcc.Graph(
#         id='example-graph',
#         figure=fig
#     )
# ])

In [6]:
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [7]:
# test
test = df['PEEDUCA']
test.unique()

array(['Post graduate degree', 'College', 'High school or below',
       'Bachelor'], dtype=object)

[0, 1]
<class 'list'>
                 TELFS               PEEDUCA  count
0             Employed              Bachelor  34532
1             Employed               College  39839
2             Employed  High school or below  45074
3             Employed  Post graduate degree  21682
4   Not in labor force              Bachelor  10725
5   Not in labor force               College  18671
6   Not in labor force  High school or below  41456
7   Not in labor force  Post graduate degree   6308
8           Unemployed              Bachelor   1267
9           Unemployed               College   2709
10          Unemployed  High school or below   5604
11          Unemployed  Post graduate degree    588
